In [1]:

import collections as col

from emulator.env_data import high2low, fix_data
from emulator.env_factor import get_factors

from params import *


import re
import os
import numpy as np
import pandas as pd
import calendar
import matplotlib.pyplot as plt

from functools import reduce
from collections import defaultdict
from seaborn import heatmap
from copy import deepcopy
from scipy.optimize import linprog as lp
from datetime import datetime, timedelta, date
from collections import OrderedDict


#工作目录，存放代码
work_dir = os.path.dirname(os.path.abspath('__file__'))
tickdata_path = os.path.join(work_dir, 'tickdata')
factors_path = os.path.join(work_dir, 'PQFactors')
PCT_CHG_path = os.path.join(work_dir, 'PCT_CHG_NM')
concat_path = os.path.join(work_dir, 'DateConcatFactors')

def getting_factors(file_path):
    if os.path.isdir(file_path) == False:
        quotes = fix_data(os.path.join(tickdata_path, file_path))
        quotes = high2low(quotes, H2L_Freq)
        daily_quotes = high2low(quotes, D_Freq)
    
        Index = quotes.index
        High = quotes.high.values
        Low = quotes.low.values
        Close = quotes.close.values
        Open = quotes.open.values
        Volume = quotes.volume.values
        
        factors = get_factors(Index, Open, Close, High, Low, Volume, rolling = 30, drop = False)
      
        code = file_path.split('.')[0] + '.' +  file_path.split('.')[1]
        
        factors['code'] = code
        factors.to_csv(os.path.join(factors_path, file_path))
        
        pct_chg = quotes.close.pct_change()
        pct_chg.to_csv(os.path.join(PCT_CHG_path, file_path))
        
    
def concat_date_factors(file_path):
    dates = ["2013-1-1", date.today().strftime('%Y-%m-%d')]
    #dates = ["2013-1-1", "2016-6-1"]
    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
    dates_dict = OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m"), None) for _ in range((end - start).days)).keys()
    dates_series = [_ for _ in dates_dict]
    
    for _ in dates_series:
        start_date = str(_) + '-01'
        last_d = calendar.monthrange(int(_.split('-')[0]), int(_.split('-')[1]))[1]
        end_date = str(_) + '-' + str(last_d)
        st = pd.to_datetime(start_date, format = '%Y-%m-%d')
        ed = pd.to_datetime(end_date,   format = '%Y-%m-%d')
        
        factors = pd.DataFrame()
        for fpath in os.listdir(factors_path)[:10]:
            tmp = pd.read_csv(os.path.join(factors_path, fpath), encoding="gbk", engine='c')
            tmp = tmp.set_index(tmp.tradeTime)
            factor_date_index = pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d')
            
            a= (pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d') > st) &\
                    (pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d') < ed)
            
            if a.any() == False:
                continue
                
            x = tmp[(pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d') > st) &\
                    (pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d') < ed)].iloc[-1,:]
            factors = pd.concat([factors, x], axis = 1)
                
        factors = factors.T
        factors = factors.set_index(factors.code)
        _pathname = end_date.split('-')
        pathname = _pathname[0] + _pathname[1] + _pathname[2]
        factors.to_csv(os.path.join(concat_path, pathname)+'.csv', encoding='gbk')


In [2]:
#for fpath in os.listdir(tickdata_path)[:]:
#    factors = getting_factors(fpath)

In [3]:
concat_date_factors(factors_path)

601608.SH True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
600008.SH True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
600500.SH True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
300413.SZ False 2015-01-25 2013-01-01 00:00:00 2013-01-31 00:00:00
002340.SZ True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
600985.SH True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
000938.SZ True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
002131.SZ True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
600663.SH True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
601106.SH True 2013-01-06 2013-01-01 00:00:00 2013-01-31 00:00:00
601608.SH True 2013-01-06 2013-02-01 00:00:00 2013-02-28 00:00:00
600008.SH True 2013-01-06 2013-02-01 00:00:00 2013-02-28 00:00:00
600500.SH True 2013-01-06 2013-02-01 00:00:00 2013-02-28 00:00:00
300413.SZ False 2015-01-25 2013-02-01 00:00:00 2013-02-28 00:00:00
002340.SZ True 2013-01-06 2013-02-01 00:00:00 2013-02-28 00:00:00
600985.S

600985.SH True 2013-01-06 2014-03-01 00:00:00 2014-03-31 00:00:00
000938.SZ True 2013-01-06 2014-03-01 00:00:00 2014-03-31 00:00:00
002131.SZ True 2013-01-06 2014-03-01 00:00:00 2014-03-31 00:00:00
600663.SH True 2013-01-06 2014-03-01 00:00:00 2014-03-31 00:00:00
601106.SH True 2013-01-06 2014-03-01 00:00:00 2014-03-31 00:00:00
601608.SH True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
600008.SH True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
600500.SH True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
300413.SZ False 2015-01-25 2014-04-01 00:00:00 2014-04-30 00:00:00
002340.SZ True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
600985.SH True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
000938.SZ True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
002131.SZ True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
600663.SH True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
601106.SH True 2013-01-06 2014-04-01 00:00:00 2014-04-30 00:00:00
601608.SH

601608.SH True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
600008.SH True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
600500.SH True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
300413.SZ True 2015-01-25 2015-06-01 00:00:00 2015-06-30 00:00:00
002340.SZ True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
600985.SH True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
000938.SZ True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
002131.SZ True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
600663.SH True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
601106.SH True 2013-01-06 2015-06-01 00:00:00 2015-06-30 00:00:00
601608.SH True 2013-01-06 2015-07-01 00:00:00 2015-07-31 00:00:00
600008.SH True 2013-01-06 2015-07-01 00:00:00 2015-07-31 00:00:00
600500.SH True 2013-01-06 2015-07-01 00:00:00 2015-07-31 00:00:00
300413.SZ True 2015-01-25 2015-07-01 00:00:00 2015-07-31 00:00:00
002340.SZ True 2013-01-06 2015-07-01 00:00:00 2015-07-31 00:00:00
600985.SH 